# SOI1010 Machine Learning II - Assignment #1
*   **Name**: Dongmin Kim
*   **Dep**: Automotive Enginnering
*   **ID**: 2021048140
*   **Assigned**: Sep. 22, 2025
*   **Due**: Oct. 5, 2025



 # Problem #1: Multiclass Classification via k-NN on MNIST

### Setup Code

In [1]:
import numpy as np
import torch
from torchvision import datasets
trainset = datasets.MNIST(root='./data', train=True, download=True)
testset = datasets.MNIST(root='./data', train=False, download=True)

In [2]:
# Indices for train/val splits: train_idx, valid_idx
np.random.seed(0)
val_ratio = 0.1
train_size = len(trainset)
indices = list(range(train_size))
split_idx = int(np.floor(val_ratio * train_size))
np.random.shuffle(indices)
train_idx, val_idx = indices[split_idx:], indices[:split_idx]


train_data = trainset.data[train_idx].float()/255.
train_labels = trainset.targets[train_idx]

val_data = trainset.data[val_idx].float()/255.
val_labels = trainset.targets[val_idx]

test_data = testset.data.float()/255.
test_labels = testset.targets

### (a) Implement a k-NN algorithm (k = 5) using an iterative method (i.e., using for loop) to classify a single new example.

In [3]:
# Check Dimension
print('testset_data: ', testset.data.shape) # [10000, 28, 28]
print('testset_targets: ', testset.targets.shape) # [10000]
print('trainset_data: ', trainset.data.shape) # [60000, 28, 28]
print('trainset_targets: ', trainset.targets.shape) # [60000]

print('train_data: ',train_data[0].shape) # 28 x 28

testset_data:  torch.Size([10000, 28, 28])
testset_targets:  torch.Size([10000])
trainset_data:  torch.Size([60000, 28, 28])
trainset_targets:  torch.Size([60000])
train_data:  torch.Size([28, 28])


In [4]:
# train_data, train_labels
# val_data, val_labels
# test_data, test_labels

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Single Example
def iterative_method(train_data: torch.Tensor,
                     train_labels: torch.Tensor,
                     x: torch.Tensor ,
                     k: int = 5):

    num_train = train_data.shape[0]
    dists = torch.zeros(num_train)
    x_flat = x.flatten()

    for i in range(num_train):
        train_vec = train_data[i].flatten()
        diff = x_flat - train_vec
        dists[i] = (diff ** 2).sum()


    __, idx = torch.topk(dists, k=k, largest=False)
    neighbor_labels = train_labels[idx]
    pred = torch.bincount(neighbor_labels, minlength=10).argmax().item() # Number of MNIST's classes = 10

    return pred

result = iterative_method(train_data, train_labels, test_data[0])
print(result)

7


### (b) Implement a k-NN algorithm (k = 5) using the broadcasting concept you learned in the laboratory session to classify a single new example

In [5]:
# Single Example
def broadcast(train_data: torch.Tensor,
              train_labels: torch.Tensor,
              x: torch.Tensor,
              k: int = 5):

    diff = train_data - x
    dists = (diff ** 2).view(train_data.size(0), -1).sum(dim=1)
    __, idx = torch.topk(dists, k=k, largest=False)
    neighbor_labels = train_labels[idx]
    pred = torch.bincount(neighbor_labels, minlength=10).argmax().item()

    return pred

result = broadcast(train_data, train_labels, test_data[0])
print(result)

7


###  (c) Now, extend a k-NN algorithm from (b) to perform classification over all digits for all new images at once, using broadcasting (not just a single image).

In [6]:
# Knn Classifier
import torch
import torch.nn.functional as F

def knn_c(train_data: torch.Tensor,
        train_labels: torch.Tensor,
        test_data: torch.Tensor,
        k: int,
        num_classes: int = 10):

    # Flatten
    N = train_data.size(0)
    M = test_data.size(0)
    x_train = train_data.flatten(start_dim=1) # [N, D]
    x_test = test_data.flatten(start_dim=1)   # [M, D]

    # Distance
    te_norm = (x_test ** 2).sum(dim=1, keepdim=True)           # [M, 1]
    tr_norm = (x_train ** 2).sum(dim=1).unsqueeze(0)           # [1, N]
    dists = te_norm + tr_norm - 2 * (x_test @ x_train.t())  # [M, N]

    # Choosing Nearest Trainset
    __, idx = torch.topk(dists, k=k, largest=False)   # [M, k]
    labels = train_labels[idx]

    # Counting
    preds = []
    for i in range(M):
        counts = torch.bincount(labels[i], minlength=num_classes)
        preds.append(counts.argmax().item())
    return torch.tensor(preds)

    return preds

# Validation
pred_val = knn_c(train_data, train_labels, val_data, k=5)
val_acc = (pred_val == val_labels).float().mean().item()
print(f"val accuracy: {val_acc * 100:.2f}%")


val accuracy: 97.27%


### (d) If there is any issue from (c), what is the cause of the issue? Improve the algorithm from (c) by resolving the issue you find from (c) [Hint: Try to find a function to replace a part of your algorithm by either googling or going through PyTorch document].

In [7]:
# Knn Classifier
import torch
import torch.nn.functional as F

def knn_d(train_data: torch.Tensor,
            train_labels: torch.Tensor,
            test_data: torch.Tensor,
            k: int,
            num_classes: int = 10):

    # Flatten
    N = train_data.size(0)
    M = test_data.size(0)
    x_train = train_data.view(N, -1) # [N, D]
    x_test = test_data.view(M, -1)   # [M, D]

    # Distance
    dists = torch.cdist(x_test, x_train, p=2)

    # Choosing Nearest Trainset
    __, idx = torch.topk(dists, k=k, largest=False)   # [M, k]
    labels = train_labels[idx]

    # Counting
    # #### torch.bincount
    # preds = []
    # for i in range(M):
    #     counts = torch.bincount(labels[i], minlength=num_classes)
    #     preds.append(counts.argmax().item())
    # return torch.tensor(preds)

    #### scatter_add
    counts = torch.zeros(M, num_classes, dtype=torch.int64)
    counts.scatter_add_(1, labels, torch.ones_like(labels, dtype=torch.int64))
    preds = counts.argmax(dim=1)
    return preds

    # #### one_hot
    # counts = F.one_hot(labels, num_classes = num_classes)
    # counts = counts.sum(dim=1)
    # preds = counts.argmax(dim=1)

    return preds

# Validation
pred_val = knn_d(train_data, train_labels, val_data, k=5)
acc = (pred_val == val_labels).float().mean().item()
print(f"val accuracy: {acc * 100:.2f}%")

val accuracy: 97.27%


### (e) What are the hyperparameters you can tune?
Answer: k, distance metrices


### (f) Try at least two other options for each hyperparameter.


In [8]:
import torch

# Using L2 Norm
def knn_L2(train_data: torch.Tensor,
            train_labels: torch.Tensor,
            test_data: torch.Tensor,
            k: int,
            num_classes: int = 10):

    # Flatten
    N = train_data.size(0)
    M = test_data.size(0)
    x_train = train_data.view(N, -1) # [N, D]
    x_test = test_data.view(M, -1)   # [M, D]

    # Distance: L2 (squared L2 and L2 is the same result)
    dists = torch.cdist(x_test, x_train, p=2)

    # Choosing Nearest Trainset
    __, idx = torch.topk(dists, k=k, largest=False)   # [M, k]
    labels = train_labels[idx]

    # Counting
    #### scatter_add
    counts = torch.zeros(M, num_classes, dtype=torch.int64)
    counts.scatter_add_(1, labels, torch.ones_like(labels, dtype=torch.int64))
    preds_L2 = counts.argmax(dim=1)

    return preds_L2

In [9]:
# Using L1 Norm
def knn_L1(train_data: torch.Tensor,
            train_labels: torch.Tensor,
            test_data: torch.Tensor,
            k: int,
            num_classes: int = 10):

    # Flatten
    N = train_data.size(0)
    M = test_data.size(0)
    x_train = train_data.view(N, -1) # [N, D]
    x_test = test_data.view(M, -1)   # [M, D]

    # Distance: L1
    dists = torch.cdist(x_test, x_train, p=1)

    # Choosing Nearest Trainset
    __, idx = torch.topk(dists, k=k, largest=False)   # [M, k]
    labels = train_labels[idx]

    # Counting
    #### scatter_add
    counts = torch.zeros(M, num_classes, dtype=torch.int64)
    counts.scatter_add_(1, labels, torch.ones_like(labels, dtype=torch.int64))
    preds_L1 = counts.argmax(dim=1)

    return preds_L1

In [10]:
# Using cosine
def knn_cos(train_data: torch.Tensor,
            train_labels: torch.Tensor,
            test_data: torch.Tensor,
            k: int,
            num_classes: int = 10):

    # Flatten
    N = train_data.size(0)
    M = test_data.size(0)
    x_train = train_data.view(N, -1) # [N, D]
    x_test = test_data.view(M, -1)   # [M, D]

    # Distance: Cos
    dot = x_test @ x_train.t()                       # [M, N]
    norm_test = x_test.norm(dim=1, keepdim=True)     # [M, 1]
    norm_train = x_train.norm(dim=1).unsqueeze(0)    # [1, N]
    sim = dot / (norm_test * norm_train + 1e-8)      # [M, N]
    dists = 1 - sim

    # Choosing Nearest Trainset
    __, idx = torch.topk(dists, k=k, largest=False)   # [M, k]
    labels = train_labels[idx]

    # Counting
    #### scatter_add
    counts = torch.zeros(M, num_classes, dtype=torch.int64)
    counts.scatter_add_(1, labels, torch.ones_like(labels, dtype=torch.int64))
    preds_cos = counts.argmax(dim=1)

    return preds_cos

In [ ]:
# Implement
# Running Time: 1hour
# Explore the Hyperparameter
for i in range(10):
    pred_val_L1 = knn_L1(train_data, train_labels, val_data, k=i)
    pred_val_L2 = knn_L2(train_data, train_labels, val_data, k=i)
    pred_val_cos = knn_cos(train_data, train_labels, val_data, k=i)
    acc_L1 = (pred_val_L1 == val_labels).float().mean().item()
    acc_L2 = (pred_val_L2 == val_labels).float().mean().item()
    acc_cos = (pred_val_cos == val_labels).float().mean().item()
    print(f"{i} val accuracy (L1): {acc_L1 * 100:.2f}%")
    print(f"{i} val accuracy (L2): {acc_L2 * 100:.2f}%")
    print(f"{i} val accuracy (cos): {acc_cos * 100:.2f}%")




0 val accuracy (L1): 10.08%
0 val accuracy (L2): 10.08%
0 val accuracy (cos): 10.08%
1 val accuracy (L1): 96.77%
1 val accuracy (L2): 97.47%
1 val accuracy (cos): 97.97%
2 val accuracy (L1): 95.78%
2 val accuracy (L2): 96.78%
2 val accuracy (cos): 97.55%
3 val accuracy (L1): 96.93%
3 val accuracy (L2): 97.55%
3 val accuracy (cos): 97.97%
4 val accuracy (L1): 96.48%
4 val accuracy (L2): 97.27%
4 val accuracy (cos): 98.00%
5 val accuracy (L1): 96.47%
5 val accuracy (L2): 97.27%
5 val accuracy (cos): 98.02%
6 val accuracy (L1): 96.43%
6 val accuracy (L2): 97.27%
6 val accuracy (cos): 97.85%
7 val accuracy (L1): 96.60%
7 val accuracy (L2): 97.23%
7 val accuracy (cos): 97.85%
8 val accuracy (L1): 96.42%
8 val accuracy (L2): 97.20%
8 val accuracy (cos): 97.73%
9 val accuracy (L1): 96.37%
9 val accuracy (L2): 97.12%
9 val accuracy (cos): 97.67%


### (g) You can try more options if you want. What is the final test accuracy?

In [11]:
pred_test_cos = knn_cos(train_data, train_labels, test_data, k=5)
acc_test_cos = (pred_test_cos == test_labels).float().mean().item()
print(f"test accuracy (k=5, cos): {acc_test_cos * 100:.2f}%")

pred_test_L1 = knn_L1(train_data, train_labels, test_data, k=5)
acc_test_L1 = (pred_test_L1 == test_labels).float().mean().item()
print(f"test accuracy (k=5, L1): {acc_test_L1 * 100:.2f}%")

pred_test_L2 = knn_L2(train_data, train_labels, test_data, k=5)
acc_test_L2 = (pred_test_L2 == test_labels).float().mean().item()
print(f"test accuracy (k=5, L2): {acc_test_L2 * 100:.2f}%")



test accuracy (k=5, cos): 97.22%
test accuracy (k=5, L1): 96.07%
test accuracy (k=5, L2): 96.66%
